In [3]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import get_lodging_scores
import cv2  # importing cv
import imutils
import glob

#We need these in this file:
import load_read_name_extractor as lrne
import SVM_classifier_general as SVM_GENERAL
import os
import pickle

In [4]:
features_load = np.load('features/hog_features_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
names_load = np.load('features/img_names_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

FileNotFoundError: [Errno 2] No such file or directory: 'features/hog_features_64_128_hog_cells_8x8_block_2x2.npy'

In [109]:
labels = get_lodging_scores.get_labels(33)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean

print(labels[0])

[0.20888097 0.16330611 0.04696022 ... 0.06301632 0.05749686 0.03150874]
(61, 50, '160607_Min', 2.0)


In [110]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

4936


In [111]:
print(len(labels))

4936


In [112]:
names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

In [115]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame, error = SVM_GENERAL.match_pic_label_to_names(features_load, labels, names)
#data_frame_names, error_names = match_pic_label_names(features_load, labels, names) - finds images that has no labels
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


['0.26293006142559816' '0.16018666865319175' '0.05102685702749534' ...
 '0.09049865286003429' '0.12448920805047044' '1.0']


In [118]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [119]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

['1.0' '2.0' '1.0' ... '2.0' '1.0' '1.0']


SVC(class_weight='balanced')

In [120]:
print(clf)

SVC(class_weight='balanced')


In [121]:
y_pred = clf.predict(x_test)

In [122]:
filename = 'SVM_linear_64_128_3bins_cells_8x8_block_2x2_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [123]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [124]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.7197518097207859


              precision    recall  f1-score   support

         1.0       0.85      0.83      0.84       684
         2.0       0.38      0.46      0.41       193
         3.0       0.59      0.46      0.52        90

    accuracy                           0.72       967
   macro avg       0.61      0.58      0.59       967
weighted avg       0.73      0.72      0.73       967



In [ ]:
for i in range(len(names)):
    print(names[i])
for i in range(len(error)):
    print(error[i])
for i in range(len(names_load)):
    print(names_load[i])